## Tasks:
- Find information about the first dates of Non-Public Accounting jobs for each CPA Partners using a list of collected LinkedIn profiles

In [1]:
import os, random, sys, time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import random
import re

### **Since LinkedIn has a mechanism in place to prevent web scrapping that will restrict my account when suspicous activities are identified. I added some random pauses between each actions to mimic human actions and not getting blocked.

In [5]:
# Create function to perform login
def login_to_linkedin(login_info):
    # Use selenium Chrome Driver to open the login page
    browser.get('https://www.linkedin.com/uas/login')  
    file = open(login_info)
    # Read username and password saved in a text file
    lines = file.readlines()
    username = lines[0]
    password = lines[1]
    # Locate login input on the opened page and type in login info
    login_info = browser.find_element_by_id('username')
    login_info.send_keys(username)
    time.sleep(random.randint(3,10)) # pause randomly for 5-10 seconds
    login_info = browser.find_element_by_id('password')
    login_info.send_keys(password)
    time.sleep(random.randint(3,10))
    login_info.submit()

# Create function to extract information from a profile link
def get_exp_info(profile_link):
    browser.get(profile_link)
    # Scroll the end of page first then scroll to the experience section
    try:
        browser.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(random.randint(5,10))
        browser.find_element_by_id('experience-section').location_once_scrolled_into_view
    except Exception as e:
        print(e)
    finally:
    # Then find the button to expand the experience section and show all job titles
        time.sleep(random.randint(5,10)) 
        while True:
            try:
                browser.find_element_by_xpath('//li-icon[@type="chevron-down-icon"][@class="pv-profile-section__toggle-detail-icon"]').click()
            except Exception as e:
                print(e)
                break 
    
    time.sleep(random.randint(5,10))
    # Parsing source code
    src = browser.page_source
    soup = BeautifulSoup(src,'lxml')
    # Extract experience section
    exp_section = soup.find('section',{'id':'experience-section'})
    if exp_section: # If there is an experience section then do this
        exp_section = exp_section.find('ul')
        li_tags = exp_section.find_all('li')
        for i, li in enumerate(li_tags): #For each position in the list of positions do this 
            time.sleep(random.randint(5,10))
            try: #Below is the main logic
                a_tags = li.find('div').find('a')
                if not li.find('div').find_all('a'): # Dealing with multiple positions within the same company
                    comp_dirs.append('NA')
                    companies.append('NA')
                    if li.find_all('h3'):
                        if li.find('h3').find_all('span'):
                            titles.append(li.find('h3').find_all('span')[-1].get_text().strip())
                        else:
                            titles.append('NA')
                    else:
                        titles.append('NA')
                    if li.find_all('h4'):
                        if li.find('h4').find_all('span'):
                            dates.append(li.find('h4').find_all('span')[-1].get_text().strip())
                        else:
                            dates.append('Jan 1900 – Jan 1900')
                    else:
                        dates.append('Jan 1900 – Jan 1900')
                    links.append(profile_link)
                    org_types.append('NA')
                else: #Dealing with only 1 position per company
                    p_tags = a_tags.find_all('p')
                    if not p_tags:
                        href = li.find('div').find('a',href=True)['href']
                        comp_dirs.append(href)
                        if a_tags.find_all('h3'):
                            if a_tags.find('h3').find_all('span'):
                                companies.append(a_tags.find('h3').find_all('span')[-1].get_text().strip())
                            else:
                                companies.append('NA')
                        else:
                            companies.append('NA')
                        titles.append('NA')
                        dates.append('Jan 1900 – Jan 1900')
                        links.append(profile_link)
                        time.sleep(random.randint(5,10))
                        if href.startswith('/search/results') == False: #If the company has a LinkedIn profile then do this to get the type of company
                            full_link = 'https://www.linkedin.com' + href
                            browser.get(full_link)
                            src = browser.page_source
                            soup = BeautifulSoup(src,'lxml')
                            if soup.find('div',{'class':'org-top-card-summary-info-list__info-item'}):
                                org_info = soup.find('div',{'class':'org-top-card-summary-info-list__info-item'})
                                org_types.append(org_info.get_text().strip())
                            else:
                                org_types.append('NA') #If there's no profile then append 'NA'
                        else:
                            org_types.append('NA')
                    else:
                        href = li.find('div').find('a', href=True)['href']
                        comp_dirs.append(href)
                        companies.append(a_tags.find_all('p')[-1].get_text().strip().split('\n')[0])
                        titles.append(a_tags.find('h3').get_text().strip())
                        if a_tags.find('h4'):
                            if a_tags.find('h4').find_all('span'):
                                dates.append(a_tags.find('h4').find_all('span')[-1].get_text().strip())
                            else:
                                dates.append('Jan 1900 – Jan 1900')
                        else:
                            dates.append('Jan 1900 – Jan 1900')
                        links.append(profile_link)
                        time.sleep(random.randint(5,10))
                        if href.startswith('/search/results') == False:
                            full_link = 'https://www.linkedin.com' + href
                            browser.get(full_link)
                            src = browser.page_source
                            soup = BeautifulSoup(src,'lxml')
                            if soup.find('div',{'class':'org-top-card-summary-info-list__info-item'}):
                                org_info = soup.find('div',{'class':'org-top-card-summary-info-list__info-item'})
                                org_types.append(org_info.get_text().strip())
                            else:
                                org_types.append('NA')
                        else:
                            org_types.append('NA')
            except: #Catching unexpected errors
                comp_dirs.append('NA')
                companies.append('NA')
                titles.append('NA')
                dates.append('Jan 1900 – Jan 1900')    
                org_types.append('NA')
                links.append(profile_link)
    else: #If no experience section then do this
        print('no experience section')
        comp_dirs.append('NA')
        companies.append('NA')
        titles.append('NA')
        dates.append('Jan 1900 – Jan 1900')    
        org_types.append('NA')
        links.append(profile_link)

In [ ]:
for set in np.arange(1,5,1): #Use more layers of random pausing in case the website has detected the existing patterns
    try:   
        for round in range(5): #We will do 5 rounds per set then take a random 5-10 minutes break
            try:
                # Compile a list of remaining profiles to be scraped
                df_results = pd.read_csv('results.csv')
                df_manual = pd.read_excel('manual results.xlsx')
                df_partners = pd.read_excel('characteristics_for_partners_2016to2018_complete3733_corporate_experience_second_half.xlsx', sheet_name='Sheet1') #Original list of profiles
                df_raw = pd.read_csv('df_raw.csv')
                completed_profiles = np.append(df_results.linkedin_profiles.unique(),df_manual.linkedin_profiles.unique())
                completed_profiles = np.append(completed_profiles, df_raw.linkedin_profiles.unique())
                df_partners = df_partners[df_partners.weblink.notna()]
                profiles = df_partners[~df_partners.weblink.isin(completed_profiles)].weblink[:10].values #Scrape 10 profiles per round
                # Create empty lists to contain scrapped data
                comp_dirs = []
                companies = []
                titles = []
                dates = []
                links = []
                org_types = []
                try_again_profiles = []
                # login to LinkedIn
                browser = webdriver.Chrome('chromedriver_win32/chromedriver')
                login_to_linkedin('login info.txt')
                time.sleep(random.randint(10,15))
                # Start scraping
                for n, p in enumerate(profiles):
                    try:
                        print('Start scraping set {} round {} profile {}'.format(set,round,p))
                        get_exp_info(p)
                        time.sleep(random.randint(3,5))
                    except Exception as e:
                        print(e)
                        print("Can't scrape profile: {}".format(p))
                        try_again_profiles.append(p)
                        try_again_text = open('try_again.txt','w') #Copy faulty profiles to a text file for further review
                        for p in try_again_profiles:
                            try_again_text.write(p + '\n')
                        try_again_text.close()
                        break
                # Save to data table
                df = pd.DataFrame(list(zip(links,comp_dirs,companies,org_types,titles,dates)),columns=['linkedin_profiles','comp_dirs','companies','org_types','titles','dates'])
                # Append to current table and print number of profiles remaining
                df_raw = df_raw.append(df, ignore_index=True)
                df_raw.to_csv('df_raw.csv', index=False)
                profiles_remaining = df_partners.weblink.nunique() - df_results.linkedin_profiles.nunique() - df_manual.linkedin_profiles.nunique() - df_raw.linkedin_profiles.nunique()
                print(profiles_remaining,'profiles remaining')
                browser.close() #Close browser after each round
                pause_period = random.randint(5,10)*60 #random 5-10 minutes break after each round
                time.sleep(pause_period)
            except Exception as e:       
                print(e)
                break #longer random pause can be placed here but since the current schedule appears to work well so there's no need for it
    except Exception as e:
        print(e)
        break